In [1]:
import tensorflow as tf
import numpy as np

In [2]:
model = tf.keras.models.load_model('/notebooks/saved_model/model2023-02-13 04:14:10.183464', compile=False)
# model = tf.keras.models.load_model('/notebooks/saved_model/model2023-02-16 14:43:58.031636', compile=False)
# model = tf.keras.models.load_model('/notebooks/saved_model/model2023-02-17 01:14:42.318862', compile=False)


2023-02-28 11:33:46.428135: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-02-28 11:33:46.428189: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-28 11:33:46.521641: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15859712 exceeds 10% of free system memory.
2023-02-28 11:33:46.539775: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15859712 exceeds 10% of free system memory.
2023-02-28 11:33:46.595459: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15859712 exceeds 10% of free system memory.
2023-02-28 11:33:50.359742: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15859712 exceeds 10% of free system memory.
2023-02-28 11:33:50.453428: W tensorflow/core/fram

In [3]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
import pathlib

val_dir = pathlib.Path('dataset/valid')
batch_size = 32
img_height = 180
img_width = 180

val_ds = tf.keras.utils.image_dataset_from_directory(
  val_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

for images, labels in val_ds.take(1):  # only take first element of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

class_names = val_ds.class_names
print(class_names)

loss, acc = model.evaluate(numpy_images, numpy_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

Found 6678 files belonging to 11 classes.
['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'healthy', 'powdery_mildew']
1/1 - 1s - loss: 0.2249 - accuracy: 0.9375 - 1s/epoch - 1s/step
Restored model, accuracy: 93.75%


In [ ]:
leaf_path = pathlib.Path('/notebooks/sample_data/IMG_4153.jpeg')
leaf_dir = pathlib.Path('/notebooks/sample_data')
for leaf in list(leaf_dir.glob('*')):
    img = tf.keras.utils.load_img(
        pathlib.Path(leaf), target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "{}: {} with a {:.2f} percent confidence."
        .format(leaf, class_names[np.argmax(score)], 100 * np.max(score))
    )
    

# img = tf.keras.utils.load_img(
#     leaf_path, target_size=(img_height, img_width)
# )
# img_array = tf.keras.utils.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Create a batch

# predictions = model.predict(img_array)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(class_names[np.argmax(score)], 100 * np.max(score))
# )

1/1 [==============================] - 0s 143ms/step
/notebooks/sample_data/IMG_4151.jpeg: Late_blight with a 88.18 percent confidence.
1/1 [==============================] - 0s 63ms/step
/notebooks/sample_data/septoria-leaf-spot-tomato-2.jpg: Septoria_leaf_spot with a 63.17 percent confidence.
1/1 [==============================] - 0s 90ms/step
/notebooks/sample_data/powderymildew1.jpg: Late_blight with a 73.49 percent confidence.
1/1 [==============================] - 0s 24ms/step
/notebooks/sample_data/bacterialspot1.jpg: Bacterial_spot with a 78.06 percent confidence.
1/1 [==============================] - 0s 68ms/step
/notebooks/sample_data/IMG_4152.jpeg: healthy with a 62.28 percent confidence.
1/1 [==============================] - 0s 23ms/step
/notebooks/sample_data/leafmold3.jpg: powdery_mildew with a 80.54 percent confidence.
1/1 [==============================] - 0s 68ms/step
/notebooks/sample_data/IMG_4198.jpeg: healthy with a 89.10 percent confidence.
1/1 [================